In [1]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

import os

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
WARNING 01-16 00:47:00 [interface.py:465] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.


In [2]:
DATASETS_DIR = "./datasets"
os.makedirs(DATASETS_DIR, exist_ok=True)

In [3]:
mbpp = load_dataset("google-research-datasets/mbpp", "sanitized", cache_dir=DATASETS_DIR)

In [4]:
os.makedirs("models", exist_ok=True)

# MODEL_NAME = "Qwen/Qwen3-0.6B"
MODEL_NAME = "Qwen/Qwen3-4B-Instruct-2507"
# MODEL_PATH = "models/qwen3-0.6b"
MODEL_PATH = "models/qwen3-4B-instruct-2507"
# OUTPUT_MODEL_DIR = "models/qwen3-0.6b-sft"
OUTPUT_MODEL_DIR = MODEL_PATH + "-sft"

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
	model_name = MODEL_PATH,
	max_seq_length=2048,
	dtype=None,
	load_in_4bit = True,
)

==((====))==  Unsloth 2026.1.3: Fast Qwen3 patching. Transformers: 4.57.3. vLLM: 0.13.0.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# 2. Навешиваем адаптеры LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 64,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
)

Unsloth 2026.1.3 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [17]:
def extract_signature_from_mbpp_code(code: str) -> str:
    # Берём первую строку, начинающуюся с def ...
    for line in code.splitlines():
        line = line.strip()
        if line.startswith("def "):
            return line.rstrip(":")  # можно оставить или убрать :
    raise ValueError("No function signature found")


def build_mbpp_prompt(example: dict) -> dict:
    # Исправлено: -> dict, так как возвращаем словарь
    task_text = example["prompt"]
    code_solution = example["code"]

    # Предположу, что эта функция импортирована или определена где-то рядом
    signature_line = extract_signature_from_mbpp_code(example["code"])

    system_msg = (
        "You are an expert Python coding assistant. "
        "Given a problem description and function signature, "
        "implement the function body so that it passes all tests."
    )

    user_msg = (
        "Problem:\n"
        f"{task_text}\n\n"
        "Use the following function signature:\n"
        f"{signature_line}:\n\n"
        "Write the full Python function implementation. "
        "Do NOT change the function name or arguments. "
        "Return only Python code."
    )

    assistant_msg = (
        f"```python\n{code_solution}\n```"
    )

    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg},
        {"role": "assistant", "content": assistant_msg},
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False,
    )

    return {"text": prompt}

In [18]:
dataset = mbpp.map(build_mbpp_prompt)
dataset

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/257 [00:00<?, ? examples/s]

Map:   0%|          | 0/43 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source_file', 'task_id', 'prompt', 'code', 'test_imports', 'test_list', 'text'],
        num_rows: 120
    })
    test: Dataset({
        features: ['source_file', 'task_id', 'prompt', 'code', 'test_imports', 'test_list', 'text'],
        num_rows: 257
    })
    validation: Dataset({
        features: ['source_file', 'task_id', 'prompt', 'code', 'test_imports', 'test_list', 'text'],
        num_rows: 43
    })
    prompt: Dataset({
        features: ['source_file', 'task_id', 'prompt', 'code', 'test_imports', 'test_list', 'text'],
        num_rows: 7
    })
})

In [19]:
dataset["train"][0]

{'source_file': 'Benchmark Questions Verification V2.ipynb',
 'task_id': 602,
 'prompt': 'Write a python function to find the first repeated character in a given string.',
 'code': 'def first_repeated_char(str1):\n  for index,c in enumerate(str1):\n    if str1[:index+1].count(c) > 1:\n      return c',
 'test_imports': [],
 'test_list': ['assert first_repeated_char("abcabc") == "a"',
  'assert first_repeated_char("abc") == None',
  'assert first_repeated_char("123123") == "1"'],
 'text': '<|im_start|>system\nYou are an expert Python coding assistant. Given a problem description and function signature, implement the function body so that it passes all tests.<|im_end|>\n<|im_start|>user\nProblem:\nWrite a python function to find the first repeated character in a given string.\n\nUse the following function signature:\ndef first_repeated_char(str1):\n\nWrite the full Python function implementation. Do NOT change the function name or arguments. Return only Python code.<|im_end|>\n<|im_start|

In [20]:
# 4. Тренер
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    dataset_text_field = "text",
    max_seq_length = 2048,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        max_steps = 100, # Обучаем немного для теста
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        output_dir = "checkpoints",
        optim = "adamw_8bit",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/120 [00:00<?, ? examples/s]

In [21]:
print("Начинаем обучение...")
trainer.train()

# 5. СОХРАНЕНИЕ (Самое важное для твоего скрипта)
# Мы сливаем веса (Merge) и сохраняем в формате, понятном vLLM
print(f"Сохраняем объединенную модель в {OUTPUT_MODEL_DIR}...")
model.save_pretrained_merged(OUTPUT_MODEL_DIR, tokenizer, save_method="merged_16bit")
print("Готово! Теперь запускай свой скрипт инференса.")

The model is already on multiple devices. Skipping the move to device specified in `args`.


Начинаем обучение...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 120 | Num Epochs = 13 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 132,120,576 of 4,154,588,672 (3.18% trained)


Step,Training Loss
10,1.165400
20,0.397000
30,0.296700
40,0.175600
50,0.096000
60,0.063800
70,0.045300
80,0.038300
90,0.036300
100,0.033900


Сохраняем объединенную модель в models/qwen3-4B-instruct-2507-sft...
Detected local model directory: /home/pavel/projects/mouse-learning/cl-rl-course-work/models/qwen3-4B-instruct-2507
Found HuggingFace hub cache directory: /home/pavel/.cache/huggingface/hub


Unsloth: Merging weights into 16bit: 100%|██████████| 3/3 [00:31<00:00, 10.46s/it]


Unsloth: Merge process complete. Saved to `/home/pavel/projects/mouse-learning/cl-rl-course-work/models/qwen3-4B-instruct-2507-sft`
Готово! Теперь запускай свой скрипт инференса.
